In [5]:
from __future__ import unicode_literals, print_function
import os
import numpy as np 
import spacy, csv
# from textstat.textstat import textstatistics, easy_word_set, egacy_round 
import nltk
import re
import io
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import PhraseMatcher

nlp = spacy.load('en')
from utilities.features_csv import to_csv
# from utilities.feature_engineering import *

In [6]:
# import os
import re 
import csv 
docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
i = 0
for root, dirs, files in os.walk("corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r', errors='ignore')
            i+=1
            text = file.read()
            docs[id] = text

        

Privacy policy
May 2018

PRIVACY POLICY
Effective Date: 21 May 2018

1.

INTRODUCTION

This Privacy Policy sets out the basis on which LMAX Broker Limited trading as LMAX Global, and LMAX Limited trading as LMAX Exchange, both being companies within the LMAX Exchange Group of Entities (LMAX, we or us or our), collects, processes and uses your personal data. This Policy applies to www.LMAX.com/global (our Website) and use of the LMAX trading platform (our Trading Platform)

We are committed to protecting your privacy and personal information. Please read this Privacy Policy carefully to ensure you understand our views and practices regarding your personal data and how we will treat it. If you have any questions you can contact us using the details below. Please also refer to our Cookie Policy [https://www.lmax.com/pdf/Cookies-Policy.pdf] which sets out the use of cookies and other web tracking technology via our website.

2.

INFORMATION WE MAY COLLECT

We may collect the following per